In [23]:
#import sys
#!conda install --yes -c conda-forge  pyvista
#!conda install --yes  matplotlib
#!conda install -c conda-forge pynrrd 
#!conda install -c conda-forge ipywidgets
#!conda install -c anaconda scipy

^C


In [1]:
import numpy as np
dtype = np.dtype('B')
display_data = None

ORIG_SIZE = 128

try:
    ds = np.DataSource(None)
    gfile = ds.open('http://cdn.klacansky.com/open-scivis-datasets/hydrogen_atom/hydrogen_atom_128x128x128_uint8.raw')
    display_data = np.fromfile(gfile,dtype)
        
except IOError:
    print("Error opening file")


In [2]:
d2_raw = np.array(display_data).reshape((ORIG_SIZE,ORIG_SIZE,ORIG_SIZE))

In [ ]:
max_val = 0;

factor = 1
SIZE = int(ORIG_SIZE/factor)

d2 = np.zeros((SIZE,SIZE,SIZE))


for i, plane in enumerate(d2):
    if i%factor != 0:
        continue
    for j,row in enumerate(plane):
        if j%factor != 0:
            continue
        for k, pix in enumerate(row):
            if j%factor != 0:
                continue
            max_val = max(max_val,pix)
            print("setting", i//factor, j//factor, k//factor, "to", d2[i][j][k])
            d2[i//factor][j//factor][k//factor] = d2[i][j][k]
            
print(max_val)

In [8]:
%matplotlib inline
from matplotlib import pyplot as plt


gamma_base = 0.2
gamma_steps = 20
gamma_step_size = 0.2

# # gamma_base = 0.01
# # gamma_steps = 15
# # gamma_step_size = 0.05

# # low_pass_value = 0
# # high_pass_value = 255 
# # gamma_base = 0.5
# # gamma_steps = 20
# # gamma_step_size = 1
# gamma_lookup = np.zeros((gamma_steps,256))

# raycast_display = np.zeros((gamma_steps,128,128))

# #d2 = np.rot90(d2,axes=[0,2])

# for i, plane in enumerate(d2):
#     print("Plane: ",i)
#     for j,row in enumerate(plane):
#         for k, pix in enumerate(row):
#             if (pix <= low_pass_value):
#                 continue
            
#             if (pix >= high_pass_value):
#                 continue

#             scale = 1 # 255 / high_pass_value
#             pix0 = min(255,int( pix * scale))
            
#             if(pix0 > 255):
#                 print ("Out of bounds", pix, pix0, scale, high_pass_value)
            
#             for l in range(gamma_steps):
               
#                 if(raycast_display[l][j][k] > 0.99):
#                     continue
                    
#                 g_pix = None

#                 if g_pix == 0 or gamma_lookup[l][pix0] != 0:
#                     g_pix = gamma_lookup[l][pix0]
#                 else:
#                     g_pix = (pix/255)**(gamma_base + l*gamma_step_size)
#                     gamma_lookup[l][pix0] = g_pix

#                 raycast_display[l][j][k] += g_pix*(1-raycast_display[l][j][k])
# #                 raycast_display[l][j][k] += (pix/(255*(gamma_base + l*gamma_step_size)))*(1-raycast_display[l][j][k])

    

In [13]:
raster_display = np.zeros((SIZE,SIZE))
def calculate_image(gamma,scale,low_pass_value,high_pass_value):
    print("Gamma",gamma,"scale",scale)

    raster_display = np.zeros((SIZE,SIZE))
    _gamma_lookup = np.zeros((256))

    #d2 = np.rot90(d2,axes=[0,2])

    for i, plane in enumerate(d2):
        #print("Plane: ",i)
        for j,row in enumerate(plane):
            for k, pix in enumerate(row):
                print("row",row)
                if (pix <= low_pass_value):
                    continue

                if (pix >= high_pass_value):
                    continue

                pix0 = min(255,int( pix * scale))

                if(pix0 > 255):
                    print ("Out of bounds", pix, pix0, scale, high_pass_value)

                if(raster_display[j][k] > 0.995):
                    print("max value at",i,j)
                    continue

                g_pix = None

                if g_pix == 0 or _gamma_lookup[pix0] != 0:
                    g_pix = _gamma_lookup[pix0]
                else:
                    g_pix = (pix0/255)**(gamma)
                    _gamma_lookup[pix0] = g_pix

                raster_display[j][k] += g_pix*(1-raster_display[j][k])
                
                if j == SIZE-1 and k == SIZE-1:
                    print(j,k," : ",  raster_display[j][k])
                
            
    print(raster_display)
#     plt.imshow(raster_display, interpolation='nearest')
#     plt.show()  
    


In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import matplotlib.animation as animation

g_gamma = 1.0
g_scale = 0.25
lp_value = 0
hp_value = 255
# def f(x):
#     plt.imshow(raycast_display[int((x-gamma_base)/gamma_step_size)], interpolation='nearest')
#     plt.show()  

# interact(f,  x=widgets.FloatSlider(min=gamma_base, step=gamma_step_size, max=gamma_base + (gamma_steps-1)*gamma_step_size, value =1))

scale_widget = widgets.FloatSlider(min=0.05, step=0.05, max=1.0, value=g_scale)
display(scale_widget)

gamma_widget =widgets.FloatSlider(min=gamma_base, step=gamma_step_size, max=gamma_base + (gamma_steps-1)*gamma_step_size, value =1)
display(gamma_widget)

lp_widget =widgets.IntSlider(min=0, step=1, max=255, value=0)
display(lp_widget)

hp_widget =widgets.IntSlider(min=10, step=1, max=255, value=255)
display(hp_widget)

fig = plt.figure()
im = plt.imshow(raster_display, animated=True)

def updatefig(*args):
    print("redrawing")
    im.set_data(raster_display)
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=5 )

plt.show()

def redraw(x):
    print(gamma_widget.value, scale_widget.value,lp_widget.value,hp_widget.value)
    calculate_image(gamma_widget.value,scale_widget.value,lp_widget.value,hp_widget.value)
    im.set_data(raster_display)
    plt.imshow(raster_display, interpolation='nearest')
    plt.show()  


#interact(f,  x=widgets.FloatSlider(min=gamma_base, step=gamma_step_size, max=gamma_base + (gamma_steps-1)*gamma_step_size, value =1))
# text = widgets.Text(
#        value='My Text',
#        description='Title', )

#interact(f,  x= widgets.Button(description='Generate'))
button =  widgets.Button(description='Generate')
button.on_click(redraw)
display(button)
                        